In [ ]:
import cv2
import json
import numpy as np

In [ ]:
# poly2mask
ann_path = '../Images/171206_054430023_Camera_5.json'
with open(ann_path, 'r') as f:
    ann = json.load(f)

h, w = ann['imageHeight'], ann['imageWidth']
ann_img = np.zeros((h, w), dtype=np.uint8)
polys = []
for poly_ann in ann['shapes']:
    if poly_ann['shape_type'] != 'polygon':
        continue
    poly = np.array(poly_ann['points']).astype(np.int64)
    polys.append(poly)
cv2.fillPoly(ann_img, polys, 255)
cv2.imwrite('../Images/171206_054430023_Camera_5.png', ann_img)

In [ ]:
# transform
img = cv2.imread('../Images/cat.png')
h, w = img.shape[:2]

# rigid
theta = 30 / 180 * np.pi
tx = 100
ty = -50
rigid_mat = np.array([[np.cos(theta), -np.sin(theta), tx],
                      [np.sin(theta),  np.cos(theta), ty]], dtype=np.float32)
rigid_img = cv2.warpAffine(img, rigid_mat, (w, h))
show_img = np.concatenate([img, rigid_img], axis=1)
cv2.imwrite('../Images/rigid.png', show_img)

# similar
theta = 30 / 180 * np.pi
tx = 50
ty = -150
s = 1.5
similar_mat = np.array([[s * np.cos(theta), -s * np.sin(theta), tx],
                        [s * np.sin(theta),  s * np.cos(theta), ty]], dtype=np.float32)
similar_img = cv2.warpAffine(img, similar_mat, (w, h))
show_img = np.concatenate([img, similar_img], axis=1)
cv2.imwrite('../Images/similar.png', show_img)

# affine
a, b, c, d = 1.1, 0.9, 0.1, 1.0
tx = -100
ty = 30
affine_mat = np.array([[a, b, tx],
                       [c, d, ty]], dtype=np.float32)
affine_img = cv2.warpAffine(img, affine_mat, (w, h))
show_img = np.concatenate([img, affine_img], axis=1)
cv2.imwrite('../Images/affine.png', show_img)

# perspective
a, b, c, d = 1.0, 0.0, 0.0, 1.0
tx = 50
ty = 0
v1 = 0
v2 = 1e-3
persp_mat = np.array([[ a,  b, tx],
                      [ c,  d, ty],
                      [v1, v2,  1]], dtype=np.float32)
persp_img = cv2.warpPerspective(img, persp_mat, (w, h))
show_img = np.concatenate([img, persp_img], axis=1)
cv2.imwrite('../Images/persp.png', show_img)

In [ ]:
# color
img = cv2.imread('../Images/cat.png')

print('contrast / brightness')
alpha = 1.0
beta = 0.0
while True:
    new_img = (img * alpha + beta)
    new_img = np.clip(new_img, 0, 255).astype(np.uint8)
    cv2.imshow('show', new_img)
    key = cv2.waitKey(1)
    if key != -1:
        print(key)
    if key == 97: # left
        beta -= 2
    if key == 100: # right
        beta += 2
    if key == 115: # down
        alpha -= 0.05
    if key == 119: # up
        alpha += 0.05
    if key == 27: # esc
        break

print('hue / saturation / value(brightness)')
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
h = 0.0
s = 0.0
v = 0.0
while True:
    new_img_hsv = img_hsv + np.array([h, s, v]).reshape(1, 1, 3)
    new_img_hsv = np.clip(new_img_hsv, 0, 255).astype(np.uint8)
    new_img = cv2.cvtColor(new_img_hsv, cv2.COLOR_HSV2BGR)
    cv2.imshow('show', new_img)
    key = cv2.waitKey(1)
    if key == 97: # left
        h -= 5
    if key == 100: # right
        h += 5
    if key == 115: # down
        s -= 5
    if key == 119: # up
        s += 5
    if key == 113: # q
        v -= 5
    if key == 101: # e
        v += 5
    if key == 27: # esc
        break

print('equalization')
b = cv2.equalizeHist(img[..., 0])
g = cv2.equalizeHist(img[..., 1])
r = cv2.equalizeHist(img[..., 2])
new_img = np.stack([b, g, r], axis=-1)
cv2.imshow('show', new_img)
key = cv2.waitKey(0)

In [ ]:
img = cv2.imread('../Images/cat.png')

print('blur')
new_img = cv2.blur(img, (5, 5))
show_img = np.concatenate([img, new_img], axis=1)
cv2.imshow('show', show_img)
cv2.waitKey(0)

print('sharpen')
new_img = cv2.blur(img, (5, 5))
new_img = cv2.addWeighted(img, 1.5, new_img, -0.5, 0)
show_img = np.concatenate([img, new_img], axis=1)
cv2.imshow('show', show_img)
cv2.waitKey(0)

print('gaussian noise')
mean = 0
std = 50
noise = np.random.randn(*img.shape) * std
new_img = np.clip((img.astype(np.float32) + noise), 0, 255).astype(np.uint8)
show_img = np.concatenate([img, new_img], axis=1)
cv2.imshow('show', show_img)
cv2.waitKey(0)

print('poisson noise')
crop_img = img[100:200, 100:200]
lam = 100.0
noise = np.random.poisson(lam=lam, size=crop_img.shape) - lam
new_img = np.clip(crop_img.astype(np.float32) + noise, 0, 255).astype(np.uint8)
show_img = np.concatenate([crop_img, new_img], axis=1)
show_img = cv2.resize(show_img, (1000, 500))
cv2.imshow('show', show_img)
cv2.waitKey(0)

print('salt pepper noise')
rate = 0.02
noise = np.random.choice((0, 256, -256), (img.shape[0], img.shape[1]), p=(1-rate, rate/2, rate/2))
noise = np.expand_dims(noise, axis=-1)
new_img = np.clip(img.astype(np.float32) + noise, 0, 255).astype(np.uint8)
show_img = np.concatenate([img, new_img], axis=1)
cv2.imshow('show', show_img)
cv2.waitKey(0)